<a href="https://colab.research.google.com/github/annbgrge/Data-Analytics-for-Disaster-Management-Response/blob/main/Nepal_Tweets_DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tweets.csv',names=['data'])


In [ ]:
for col in df:
    df[col] =df[col].astype(str).str.replace("(","").str.replace(")","")

<ipython-input-5-9f076349d8c7>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] =df[col].astype(str).str.replace("(","").str.replace(")","")


In [ ]:
df = df['data'].str.split(',', expand=True)

In [ ]:
df = df.fillna(value=" ")

In [ ]:
df["texts"] = df[2].map(str)+df[3].map(str)+df[4].map(str)+df[5].map(str)+df[6].map(str)+df[7].map(str)+df[8].map(str)

In [ ]:
df = df.drop(columns=[2,3,4,5,6,7,8],axis = 1)

In [ ]:
df.head()

,0,1,texts
0,0,'591764470197723137','#Nepal #news Adding to Bhutanese Refugees Bh...
1,0,'591779538046218240',"""The massacre of people's movement says peace..."
2,0,'591728244925210624',"""I'm lonely maybe you're going to die without..."
3,0,'591774433926193152','#nepal #news 6 billion collection collection...
4,0,'591764472651386882','#Nepal #news Indian proposal to crash airpor...


In [ ]:
df.columns = ['TweetClass','TweetId','Tweet']

In [ ]:
df = df.drop(['TweetId'], axis=1)

In [ ]:
df.shape

(18226, 2)

In [ ]:
df['TweetClass'].value_counts()

0    17370
2      658
1      198
Name: TweetClass, dtype: int64

#**Data Cleaning**
---

In [ ]:
df.duplicated().sum()

566

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
df['TweetClass'].value_counts()

0    16828
2      641
1      191
Name: TweetClass, dtype: int64

#**DistilBERT CLASSIFICATION**
---

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.3 MB/s eta 0:00:00


In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd
import json
import gc

In [ ]:
df['encoded_cat'] = df['TweetClass'].astype('category').cat.codes
df.head()

,TweetClass,Tweet,encoded_cat
0,0,'#Nepal #news Adding to Bhutanese Refugees Bh...,0
1,0,"""The massacre of people's movement says peace...",0
2,0,"""I'm lonely maybe you're going to die without...",0
3,0,'#nepal #news 6 billion collection collection...,0
4,0,'#Nepal #news Indian proposal to crash airpor...,0


In [ ]:
data_texts = df["Tweet"].to_list() # Features (not-tokenized yet)
data_labels = df["encoded_cat"].to_list() # Lables

In [ ]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=0)

# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=10, batch_size=16,
          validation_data=val_dataset.shuffle(1000).batch(16))

Epoch 1/10
875/875 [==============================] - 310s 317ms/step - loss: 0.1318 - accuracy: 0.9616 - val_loss: 0.0930 - val_accuracy: 0.9697
Epoch 2/10
875/875 [==============================] - 270s 309ms/step - loss: 0.0715 - accuracy: 0.9755 - val_loss: 0.0903 - val_accuracy: 0.9649
Epoch 3/10
875/875 [==============================] - 264s 301ms/step - loss: 0.0452 - accuracy: 0.9855 - val_loss: 0.0876 - val_accuracy: 0.9734
Epoch 4/10
875/875 [==============================] - 263s 301ms/step - loss: 0.0328 - accuracy: 0.9891 - val_loss: 0.1228 - val_accuracy: 0.9723
Epoch 5/10
875/875 [==============================] - 270s 308ms/step - loss: 0.0254 - accuracy: 0.9920 - val_loss: 0.1334 - val_accuracy: 0.9717
Epoch 6/10
875/875 [==============================] - 270s 308ms/step - loss: 0.0199 - accuracy: 0.9938 - val_loss: 0.1353 - val_accuracy: 0.9754
Epoch 7/10
875/875 [==============================] - 270s 308ms/step - loss: 0.0158 - accuracy: 0.9954 - val_loss: 0.1300 -

In [ ]:
save_directory = "drive/MyDrive/Nepal/distilBERT_Models" # change this to your preferred location

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('drive/MyDrive/Nepal/distilBERT_Models/tokenizer_config.json',
 'drive/MyDrive/Nepal/distilBERT_Models/special_tokens_map.json',
 'drive/MyDrive/Nepal/distilBERT_Models/vocab.txt',
 'drive/MyDrive/Nepal/distilBERT_Models/added_tokens.json')

In [ ]:
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at drive/MyDrive/Nepal/distilBERT_Models were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at drive/MyDrive/Nepal/distilBERT_Models and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
